# CMR Resource Configurator

In [4]:
from setvar import *

In [2]:
import re, os, sys

In [3]:
setvar("PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin")

PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin


## Step 1: Select your tenant

In [4]:
!tenants-list

3dem
agave.prod
araport.org
designsafe
iplantc.org
irec
portals
sd2e
sgci
tacc.prod
vdjserver.org


In [5]:
!tenants-init -t tacc.prod

You are now configured to interact with the APIs at https://api.tacc.utexas.edu/


## Step 2: Set your Agave Username and Password

In [6]:
setvar("AGAVE_USERNAME=tg457049")

AGAVE_USERNAME=tg457049


In [7]:
#os.remove("AGAVE_PASSWD.txt")
readpass("AGAVE_PASSWD")

Password or secret: AGAVE_PASSWD
Reading file `AGAVE_PASSWD.txt'


## Step 3: Provide Information about the Resource
Here you should supply the fully qualified domain name
of the machine, the port it will be accessed on, the
user that will be accessing the machine, etc.

In [37]:
setvar("""
MACHINE_USERNAME=sbrandt
MACHINE_FULL=rostam.cct.lsu.edu
PORT=8000
ALLOCATION=hpc_startup_funwave
BASE_APP_NAME=crcollaboratory
WORK_DIR=work
ROOT_DIR=/home/${MACHINE_USERNAME}/root
HOME_DIR=agave
SCRATCH_DIR=scratch
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq
""")

MACHINE_USERNAME=sbrandt
MACHINE_FULL=rostam.cct.lsu.edu
PORT=8000
ALLOCATION=hpc_startup_funwave
BASE_APP_NAME=crcollaboratory
WORK_DIR=work
ROOT_DIR=/home/sbrandt/root
HOME_DIR=agave
SCRATCH_DIR=scratch
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq


## Step 4: Provide the login credentials

In [27]:
#os.remove("MACHINE_PASSWD.txt")
readpass("MACHINE_PASSWD")

Password or secret: MACHINE_PASSWD
········
Writing file `MACHINE_PASSWD.txt'


In [28]:
# Parse out the name of the machine from its domain
g = re.match(r'(\w+)\.(.*)',os.environ["MACHINE_FULL"])
setvar("""
MACHINE={}
DOMAIN={}
""".format(g.group(1),g.group(2)))

MACHINE=rostam
DOMAIN=cct.lsu.edu


In [29]:
setvar("""
APP_NAME=${BASE_APP_NAME}-${MACHINE}-${AGAVE_USERNAME}
VER_APP_NAME=${APP_NAME}-2.0
STORAGE_MACHINE=${MACHINE}-storage-${AGAVE_USERNAME}
EXEC_MACHINE=${MACHINE}-exec-${AGAVE_USERNAME}
""")

APP_NAME=crcollaboratory-rostam-tg457049
VER_APP_NAME=crcollaboratory-rostam-tg457049-2.0
STORAGE_MACHINE=rostam-storage-tg457049
EXEC_MACHINE=rostam-exec-tg457049


In [30]:
!clients-delete -u $AGAVE_USERNAME -p $AGAVE_PASSWD $APP_NAME
!clients-create -p $AGAVE_PASSWD -S -N $APP_NAME -u $AGAVE_USERNAME
!auth-tokens-create -u $AGAVE_USERNAME -p $AGAVE_PASSWD

Successfully deleted client crcollaboratory-rostam-tg457049
Successfully created client crcollaboratory-rostam-tg457049
key: 2Af2PYqQbqjxKYO__TmT3jWNPeYa 
secret: FiybFGgezgKz0XvrF5TeBBEKGBIa
Token for tacc.prod:tg457049 successfully refreshed and cached for 14400 seconds
47647d95d465fea65b48bca1dba11d


In [12]:
%%bash
if [ ! -r ~/.ssh/id_rsa ]
then
  mkdir -p ~/.ssh
  chmod 700 ~/.ssh
  ssh-keygen -N '' -f ~/.ssh/id_rsa
fi
jsonpki --public ~/.ssh/id_rsa.pub > ./id_rsa.pub.txt
jsonpki --private ~/.ssh/id_rsa > ./id_rsa.txt
chmod 600 ./id_rsa.txt

In [33]:
os.environ["PUB_KEY"] = readfile("id_rsa.pub.txt")
os.environ["PRIV_KEY"] = readfile("id_rsa.txt")

Reading file `id_rsa.pub.txt'
Reading file `id_rsa.txt'


In [38]:
writefile("${STORAGE_MACHINE}.txt","""{
    "id": "${STORAGE_MACHINE}",
    "name": "${MACHINE} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE} computer",
    "site": "${DOMAIN}",
    "type": "STORAGE",
    "storage": {
       "host": "${MACHINE_FULL}",
       "port": ${PORT},
       "protocol": "SFTP",
       "rootDir": "${ROOT_DIR}",
       "homeDir": "${HOME_DIR}",
       "auth": {
           "username" : "${MACHINE_USERNAME}",
           "publicKey" : "${PUB_KEY}",
           "privateKey" : "${PRIV_KEY}",
           "type" : "SSHKEYS"
        }
    }
}
""")

Writing file `rostam-storage-tg457049.txt'


In [39]:
!systems-addupdate -F ${STORAGE_MACHINE}.txt

Successfully added system rostam-storage-tg457049


### List files on the STORAGE_MACHINE to ensure access is correctly configured.

In [40]:
!files-list -S ${STORAGE_MACHINE} ./ | head -5

.
foo.txt


## Step 5: More Machine Details
In order to properly use the execution machine, you will
need to provide agave with details about its queuing system
as well as the default queue that jobs will be run in. Please
edit the next section carefully.

In [41]:
# Configure information about the machine/queue
setvar("""
QUEUE=none
MAX_TIME=72:00:00
NODES=1
PROCS=10
MAX_JOBS=3
""")
 
writefile("${EXEC_MACHINE}.txt","""
{
    "id": "${EXEC_MACHINE}",
    "name": "${MACHINE} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE} computer",
    "site": "${DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "CLI",
    "scheduler" : "FORK",
    "environment": null,
    "scratchDir" : "${SCRATCH_DIR}",
    "queues": [
       {
            "name": "${QUEUE}",
            "default": true,
            "maxJobs": ${MAX_JOBS},
            "maxUserJobs": ${MAX_JOBS},
            "maxNodes": ${NODES},
            "maxProcessorsPerNode": ${PROCS},
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "${MAX_TIME}"
        }
    ],
    "login": {
       "auth": {
           "username" : "${MACHINE_USERNAME}",
           "publicKey" : "${PUB_KEY}",
           "privateKey" : "${PRIV_KEY}",
           "type" : "SSHKEYS"
        },
        "host": "${MACHINE_FULL}",
        "port": ${PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${MACHINE_FULL}",
        "port": ${PORT},
        "protocol": "SFTP",
        "rootDir": "${ROOT_DIR}",
        "homeDir": "${HOME_DIR}",
        "auth": {
           "username" : "${MACHINE_USERNAME}",
           "publicKey" : "${PUB_KEY}",
           "privateKey" : "${PRIV_KEY}",
           "type" : "SSHKEYS"
         }
     }
    },
    "workDir": "${WORK_DIR}"
}""")                        

!systems-addupdate -F ${EXEC_MACHINE}.txt

QUEUE=none
MAX_TIME=72:00:00
NODES=1
PROCS=10
MAX_JOBS=3
Writing file `rostam-exec-tg457049.txt'
Successfully added system rostam-exec-tg457049


### List files on the EXEC_MACHINE to ensure access is correctly configured.

In [42]:
!files-list -S ${EXEC_MACHINE} ./ | head -5

.
foo.txt


### Create the batch script used to run jobs. This should not need editing.

In [43]:
writefile("${APP_NAME}-wrapper.txt","""
#!/bin/bash
handle_trap() {
  rc=\$?
  if [ "\$rc" != 0 ]
  then
    \$(\${AGAVE_JOB_CALLBACK_FAILURE})
  fi
}

trap 'handle_trap' ERR EXIT
echo 'running \${simagename} model'
# Setting the x flag will echo every
# command onto stderr. This is
# for debugging, so we can see what's
# going on.
set -x
set -e
echo ==PWD=============
# We also print out the execution
# directory. Again, for debugging purposes.
pwd
echo ==JOB=============

# Use a global nodefile on this machine
LOCAL_NODEFILE=/home/funwave/nodefile.txt
cp \$LOCAL_NODEFILE \$PWD/nodes.txt

# Prepare the nodes to run the image
# The ssh software is configured to export
# the following two variables which will be
# used to control the singularity iamge.
export SING_OPTS="--bind \$PWD:/workdir \$SING_OPTS"
export SING_IMAGE=/home/sbrandt/images/\${simagename}.simg

export NP=\${AGAVE_JOB_PROCESSORS_PER_NODE}

tar xzvf input.tgz

mkdir -p output

/usr/local/bin/singularity exec \$SING_OPTS \$SING_IMAGE bash /usr/local/bin/runapp.sh
mv input/* output/
rm -f output/PRINT*
tar cvzf output.tar.gz output
""")

!files-mkdir -S ${STORAGE_MACHINE} -N ${DEPLOYMENT_PATH}
!files-mkdir -S ${STORAGE_MACHINE} -N inputs
!files-upload -F ${APP_NAME}-wrapper.txt -S ${STORAGE_MACHINE} ${DEPLOYMENT_PATH}/

writefile("test.txt","""
parfile="input.txt"
${APP_NAME}-wrapper.txt
""")

!files-mkdir -S ${STORAGE_MACHINE} -N ${DEPLOYMENT_PATH}
!files-upload -F test.txt -S ${STORAGE_MACHINE} ${DEPLOYMENT_PATH}/

writefile("${APP_NAME}.txt","""
{  
    "name":"${APP_NAME}",
    "version":"2.0",
    "label":"${APP_NAME}",
    "shortDescription":"Run ISAAC app",
    "longDescription":"",
    "deploymentSystem":"${STORAGE_MACHINE}",
    "deploymentPath":"${DEPLOYMENT_PATH}",
    "templatePath":"${APP_NAME}-wrapper.txt",
    "testPath":"test.txt",
    "executionSystem":"${EXEC_MACHINE}",
    "executionType":"CLI",
    "parallelism":"PARALLEL",
    "allocation":"${ALLOCATION}",
    "modules":[],
    "inputs":[
        {   
        "id":"input tarball",
        "details":{  
            "label":"input tarball",
            "description":"",
            "argument":null,
            "showArgument":false
        },
        "value":{  
            "default":"",
            "order":0,
            "required":false,
            "validator":"",
            "visible":true
        }
    }   

],
"parameters":[
{
  "id": "simagename",
  "value": {
    "visible": true,
    "required": false,
    "type": "string",
    "order": 0,
    "enquote": false,
    "default": "python",
    "validator": null
  },
  "details": {
    "label": "Singularity Image",
    "description": "The Singularity image to run: swan, funwave",
    "argument": null,
    "showArgument": false,
    "repeatArgument": false
  },
  "semantics": {
    "minCardinality": 0,
    "maxCardinality": 1,
    "ontology": []
  }
}
],
"outputs":[  
    {  
        "id":"Output",
        "details":{  
            "description":"The output",
            "label":"tables"
        },
        "value":{  
            "default":"",
            "validator":""
        }
    }
  ]
}
""")


!apps-addupdate -F ${APP_NAME}.txt

print ("Successfully configured Agave")

Writing file `crcollaboratory-rostam-tg457049-wrapper.txt'
Successfully created folder agave-deployment
Successfully created folder inputs
Uploading crcollaboratory-rostam-tg457049-wrapper.txt...
######################################################################### 100.0%
Writing file `test.txt'
Successfully created folder agave-deployment
Uploading test.txt...
######################################################################### 100.0%
Writing file `crcollaboratory-rostam-tg457049.txt'
Successfully added app crcollaboratory-rostam-tg457049-2.0
Successfully configured Agave


In [44]:
import json, os
from command import cmd
metadata = {"name":os.environ["EXEC_MACHINE"]+"::queue","value":os.environ["QUEUE"]}
c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
found = False
for k in c["stdout"]:
    ks = k.strip()
    if ks != '':
        cmd(["metadata-addupdate","-F","-",ks],inputs=json.dumps(metadata))
        found = True
        break
if not found:
    cmd(["metadata-addupdate","-F","-"],inputs=json.dumps(metadata))

cmd: metadata-list -Q {"name":"rostam-exec-tg457049::queue"}

cmd: metadata-addupdate -F -
Successfully submitted metadata object 1070943292343980521-242ac119-0001-012


## Step 6: Grant Access
By default, no one will be able to use the resource you configure unless you specifically grant them access.
The following two functions can be used to grant or revoke access to an Agave user.

In [22]:
# The following two commands enable you to grant or revoke the ability
# to use a given application to a given user
from command import cmd
def grant_user(user):
    cmd("apps-pems-update -u {u} -p READ_EXECUTE $VER_APP_NAME".format(u=user))
    cmd("systems-roles-addupdate -r USER -u {u} $STORAGE_MACHINE".format(u=user))
    cmd("systems-roles-addupdate -r USER -u {u} $EXEC_MACHINE".format(u=user))
    c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
    for line in c["stdout"]:
        line = line.strip()
        if line != '':
            cmd(["metadata-pems-addupdate","-u",user,"-p","READ",line])
def revoke_user(user):
    cmd("apps-pems-update -u {u} -p NONE $VER_APP_NAME".format(u=user))
    cmd("systems-roles-addupdate -r NONE -u {u} $STORAGE_MACHINE".format(u=user))
    cmd("systems-roles-addupdate -r NONE -u {u} $EXEC_MACHINE".format(u=user))
    c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
    for line in c["stdout"]:
        line = line.strip()
        if line != '':
            cmd(["metadata-pems-addupdate","-u",user,"-p","NONE",line])

In [ ]:
!auth-tokens-refresh

In [25]:
grant_user("ysboss")

cmd: apps-pems-update -u ysboss -p READ_EXECUTE crcollaboratory-melete05-tg457049-2.0
Successfully updated permission for ysboss
cmd: systems-roles-addupdate -r USER -u ysboss melete05-storage-tg457049
Successfully updated roles for user ysboss on melete05-storage-tg457049
cmd: systems-roles-addupdate -r USER -u ysboss melete05-exec-tg457049
Successfully updated roles for user ysboss on melete05-exec-tg457049
cmd: metadata-list -Q {"name":"melete05-exec-tg457049::queue"}
997330297259552280-242ac119-0001-012
cmd: metadata-pems-addupdate -u ysboss -p READ 997330297259552280-242ac119-0001-012
Successfully updated permission for ysboss


In [26]:
revoke_user("syuan")

cmd: apps-pems-update -u syuan -p NONE crcollaboratory-melete05-tg457049-2.0
Successfully updated permission for syuan
cmd: systems-roles-addupdate -r NONE -u syuan melete05-storage-tg457049
Successfully updated roles for user syuan on melete05-storage-tg457049
cmd: systems-roles-addupdate -r NONE -u syuan melete05-exec-tg457049
Successfully updated roles for user syuan on melete05-exec-tg457049
cmd: metadata-list -Q {"name":"melete05-exec-tg457049::queue"}
997330297259552280-242ac119-0001-012
cmd: metadata-pems-addupdate -u syuan -p NONE 997330297259552280-242ac119-0001-012
Successfully updated permission for syuan
